# Reloading

Sometimes vehicles can execute multiple trips during their shift, by reloading at the depot when needed.
This effectively mitigates the capacity constraint we have so far seen, because vehicles can always choose to return to the depot to reload if needed.
PyVRP supports a very general form of reloading, with free depot selection from a list of alternatives.
Optionally, the maximum number of reloads per vehicle type may also be restricted.

In [ ]:
import pyvrp
import pyvrp.plotting
import pyvrp.stop

We will investigate a simple example, with a single vehicle having a capacity of 15.
Without reloading, the clients demand more than the vehicle can carry.
By reloading at the depot, the vehicle can service all client demand in two trips. 

In [ ]:
COORDS = [(456, 320), (312, 418), (114, 80), (570, 160), (684, 240)]
DEMANDS = [5, 8, 7, 6]

In [ ]:
m = pyvrp.Model()

depot = m.add_depot(x=COORDS[0][0], y=COORDS[0][1])
m.add_vehicle_type(
    1,
    capacity=15,
    reload_depots=[depot],  # where reloads may take place
    max_reloads=2,  # maximum number of reload depot visits on a route
)

for idx in range(1, len(COORDS)):
    m.add_client(
        x=COORDS[idx][0],
        y=COORDS[idx][1],
        delivery=DEMANDS[idx - 1],
    )

for frm in m.locations:
    for to in m.locations:
        distance = abs(frm.x - to.x) + abs(frm.y - to.y)
        m.add_edge(frm, to, distance=distance)

res = m.solve(stop=pyvrp.stop.MaxRuntime(1))

Returns to a reload depot are marked with a `|`, as follows:

In [ ]:
print(res.best)

Let's investigate the route in more detail:

In [ ]:
route = res.best.routes()[0]

for idx, trip in enumerate(route.trips()):
    print(f"- Trip #{idx + 1}:")
    print(f"   | Client visits: {trip.visits()}.")
    print(f"   | Distance: {trip.distance()}.")
    print(f"   | Load: {trip.delivery()}.")

A plot reveals the routing and reloading decisions that PyVRP determined:

In [ ]:
pyvrp.plotting.plot_solution(res.best, m.data(), plot_clients=True)

## Conclusion

You now know how to use PyVRP's reloading feature.
PyVRP's support for reloading is very general, and is useful in many settings, like urban last-mile (return to the warehouse to pick up new goods), waste collection (to empty a full truck at a landfill or recycling centre), and more.